In [0]:
#Install prerequisite packages
!pip install python-dotenv==1.0.0

!pip install llama-index==0.10.59
!pip install llama-index-llms-openai==0.1.27
!pip install llama-index-embeddings-openai==0.1.11
!pip install llama-index-llms-azure-openai==0.1.10
!pip install llama-index-embeddings-azure-openai==0.1.11


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
INFO: pip is looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of llama-index-readers-llama-parse to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible wi

In [0]:
#Setup Azure Open AI connection
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

from llama_index.core import Settings
import os
import nest_asyncio

nest_asyncio.apply()

#API info. Replace with your own keys and end points
api_key=""
azure_endpoint=""
azure_deployment="GPT4-O"
api_version="2024-12-01-preview"

#Setup the LLM
Settings.llm=AzureOpenAI(
    model="gpt-4o",
    deployment_name=azure_deployment,
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

#Setup the embedding model RAG
Settings.embed_model= AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key='',
    azure_endpoint='',
    api_version='2023-05-15',
)

In [0]:
#Create indexes for vector search
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import  VectorStoreIndex

splitter=SentenceSplitter(chunk_size=1024)

#-------------------------------------------------------------------
#Setup Aeroflow document index
#-------------------------------------------------------------------
aeroflow_documents=SimpleDirectoryReader(
    input_files=["/Workspace/Users/yfatima1@optumcloud.com/AeroFlow_Specification_Document.pdf"])\
            .load_data()

#Read documents into nodes
aeroflow_nodes=splitter.get_nodes_from_documents(aeroflow_documents)
#Create a vector Store
aeroflow_index=VectorStoreIndex(aeroflow_nodes)
#Create a query engine
aeroflow_query_engine = aeroflow_index.as_query_engine()

[Trace(request_id=tr-62c1e8815715468cba43c91bacf79e24), Trace(request_id=tr-78e977e29c1442409cc0074c9720ce6b)]

In [0]:
#-------------------------------------------------------------------
#Setup EchoSprint document index
#-------------------------------------------------------------------
ecosprint_documents=SimpleDirectoryReader(
    input_files=["EcoSprint_Specification_Document.pdf"])\
            .load_data()
#Read documents into nodes
ecosprint_nodes=splitter.get_nodes_from_documents(ecosprint_documents)
#Create a vector Store
ecosprint_index=VectorStoreIndex(ecosprint_nodes)
#Create a query engine
ecosprint_query_engine = ecosprint_index.as_query_engine()

[Trace(request_id=tr-4450f7e63af046299673ef84c9bb870b), Trace(request_id=tr-9dfabc819c4e410ca1197b8c9862ec2e)]

In [0]:
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

#Create a query engine Tool for NoSQL
aeroflow_tool = QueryEngineTool.from_defaults(
    query_engine=aeroflow_query_engine,
    name="Aeroflow specifications",
    description=(
        "Contains information about Aeroflow : Design, features, technology, maintenance, warranty"
    ),
)

#Create a query engine Tool for NLP
ecosprint_tool = QueryEngineTool.from_defaults(
    query_engine=ecosprint_query_engine,
    name="EcoSprint specifications",
    description=(
        "Contains information about EcoSprint : Design, features, technology, maintenance, warranty"
    ),
)

#Create a Router Agent. Provide the Tools to the Agent
router_agent=RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        aeroflow_tool,
        ecosprint_tool,
    ],
    verbose=True
)

In [0]:
#Ask a question about NoSQL
response = router_agent.query("What colors are available for AeroFlow?")
print("\nResponse: ",str(response))

Selecting query engine 0: The question asks about AeroFlow, and choice 1 contains information about Aeroflow, which is relevant to the inquiry..

Response:  The AeroFlow is available in Coastal Blue, Sunset Orange, and Pearl White.


Trace(request_id=tr-abab21fe1fa243e89bc253d070e2b8bd)

In [0]:
response = router_agent.query("What colors are available for EcoSprint?")
print("\nResponse: ",str(response))

Selecting query engine 1: The question is about EcoSprint, and choice 2 contains information about EcoSprint..

Response:  The EcoSprint is available in Midnight Black, Ocean Blue, and Pearl White.


Trace(request_id=tr-c12a4eea051e482189788d89851d167e)